In [4]:
import pandas as pd
from citipy import citipy
import requests
import json
import numpy as np
import gmaps
from pprint import pprint
from config import gkey
gmaps.configure(api_key=gkey)

In [51]:
citi_data = pd.read_csv('../WeatherPy/citi_data.csv')
fig = gmaps.figure(layout = {'width':'800px', 'height':'400px'}, zoom_level=1, center =(0,0))
heat = gmaps.heatmap_layer(citi_data[['latitude','longitude']], weights=citi_data['humidity'], max_intensity=100, point_radius=5)
fig.add_layer(heat)
fig
citi_data

,Unnamed: 0,name,country,latitude,longitude,temperature,cloudiness,humidity,windspeed
0,2,shirokiy,RU,49.7580,129.5230,61.20,28.0,50.0,8.23
1,3,hasaki,JP,35.7333,140.8333,61.11,100.0,92.0,10.69
2,4,cherskiy,RU,68.7500,161.3000,62.40,90.0,30.0,7.63
3,5,ola,RU,59.5833,151.2833,42.10,100.0,77.0,4.88
4,6,dikson,RU,73.5069,80.5464,30.09,100.0,91.0,11.81
...,...,...,...,...,...,...,...,...,...
546,595,rikitea,PF,-23.1203,-134.9692,74.16,96.0,69.0,17.87
547,596,cape town,ZA,-33.9258,18.4232,54.39,75.0,92.0,6.13
548,597,arraial do cabo,BR,-22.9661,-42.0278,71.37,100.0,86.0,9.17
549,598,jamestown,SH,42.0970,-79.2353,59.61,79.0,97.0,8.41


In [6]:
ideal_temp = (citi_data['temperature']>70) & (citi_data['temperature']<80)
ideal_windspeed = citi_data['windspeed']<10
ideal_cloudiness = citi_data['cloudiness']<25
vacation_spots = citi_data[ideal_temp & ideal_windspeed & ideal_cloudiness].copy(deep=True)
vacation_spots
# a =vacation_spots.iloc[0,:]
# a[['latitude','longitude']].astype(float)



,Unnamed: 0,name,country,latitude,longitude,temperature,cloudiness,humidity,windspeed
34,41,garmsar,IR,35.2182,52.3409,74.30,13.0,35.0,3.53
56,66,hovd,MN,48.0056,91.6419,71.17,0.0,27.0,5.32
61,73,awbari,LY,26.5921,12.7805,79.54,0.0,17.0,8.05
115,130,awbari,LY,26.5921,12.7805,79.54,0.0,17.0,8.05
135,154,komsomolskiy,RU,40.4272,71.7189,75.92,9.0,35.0,2.30
138,157,komsomolskiy,RU,40.4272,71.7189,75.92,9.0,35.0,2.30
149,170,yatou,CN,37.1500,122.3833,72.48,0.0,45.0,7.29
150,171,komsomolskiy,RU,40.4272,71.7189,75.92,9.0,35.0,2.30
166,192,hovd,MN,48.0056,91.6419,71.17,0.0,27.0,5.32
190,223,yatou,CN,37.1500,122.3833,72.48,0.0,45.0,7.29


In [38]:
hotels = pd.DataFrame(columns=['name', 'city', 'country', 'latitude','longitude'])
hotels['city'] = vacation_spots['name'].str.upper()
hotels['country']=vacation_spots['country']
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
parameters ={
    'key':gkey,
    'location':'',
    'keyword':'hotel',
    'radius':5000,
}

for index,place in vacation_spots.iterrows():
    parameters['location']=f'{place["latitude"]},{place["longitude"]}'
    response = requests.get(base_url,params=parameters).json()
    try:
        hotels.loc[index,'name']=response['results'][0]['name']
        hotels.loc[index, 'latitude'] = response['results'][0]['geometry']['location']['lat']
        hotels.loc[index, 'longitude'] = response['results'][0]['geometry']['location']['lng']
    except IndexError as e:
        print(e)
hotels


,name,city,country,latitude,longitude
34,هتل شهر آفتاب Sun City,GARMSAR,IR,35.2417,52.3594
56,Steppe hotel,HOVD,MN,47.9995,91.6377
61,فندق أوباري,AWBARI,LY,26.5967,12.7661
115,فندق أوباري,AWBARI,LY,26.5967,12.7661
135,TANTANA HOTEL FERGANA,KOMSOMOLSKIY,RU,40.3968,71.7653
138,TANTANA HOTEL FERGANA,KOMSOMOLSKIY,RU,40.3968,71.7653
149,Jinyue International Hotel,YATOU,CN,37.1564,122.427
150,TANTANA HOTEL FERGANA,KOMSOMOLSKIY,RU,40.3968,71.7653
166,Steppe hotel,HOVD,MN,47.9995,91.6377
190,Jinyue International Hotel,YATOU,CN,37.1564,122.427


In [49]:
hotels.dropna(subset=['name'], inplace=True)
info_box =[ {'name':row['name'],'city':row['city'],'country':row['country']} for index,row in hotels.iterrows()]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**info) for info in info_box]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>هتل شهر آفتاب Sun City</dd>\n<dt>City</dt><dd>GARMSAR</dd>\n<dt>Country</dt><dd>IR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Steppe hotel</dd>\n<dt>City</dt><dd>HOVD</dd>\n<dt>Country</dt><dd>MN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>فندق أوباري</dd>\n<dt>City</dt><dd>AWBARI</dd>\n<dt>Country</dt><dd>LY</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>فندق أوباري</dd>\n<dt>City</dt><dd>AWBARI</dd>\n<dt>Country</dt><dd>LY</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>TANTANA HOTEL FERGANA</dd>\n<dt>City</dt><dd>KOMSOMOLSKIY</dd>\n<dt>Country</dt><dd>RU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>TANTANA HOTEL FERGANA</dd>\n<dt>City</dt><dd>KOMSOMOLSKIY</dd>\n<dt>Country</dt><dd>RU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Jinyue International Hotel</dd>\n<dt>City</dt><dd>YATOU</dd>\n<dt>Country</dt><dd>CN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>TANTANA HOTEL FERGANA</dd>\n<dt>City</dt><dd>KOMSOMOLSKIY</dd>\n<dt>Country</dt><dd>RU</dd>\n</dl>\n',
 '\n<dl>\n<dt>N

In [50]:
markers = gmaps.marker_layer(hotels[['latitude','longitude']], info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='400px', width='800px'))